In [2]:
!pip install tensorflow opencv-python sklearn matplotlib mediapipe


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [4]:
!pip install pyttsx3


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pyttsx3
engine = pyttsx3.init()
engine.say("jaffer is gay")
engine.runAndWait()

mp holistic for keypoints extraction

In [8]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [9]:
def mediapipe_det(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [10]:
def draw_stlyed_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

extracting keypoints

In [11]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

creating folders for data

In [12]:
DATA_PATH = os.path.join('MP_Data2')
actions = np.array(['hello','you','how'])
no_sequences = 30
sequence_length = 30

In [13]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass
        

data collecting

In [13]:

cap = cv2.VideoCapture(0)
time.sleep(4)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()

                image, results = mediapipe_det(frame, holistic)
                #print(results)

                
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2,cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} video number: {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1,cv2.LINE_AA)
                    cv2.waitKey(1000)

                else:
                    cv2.putText(image, f'Collecting frames for {action} video number: {sequence}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),1,cv2.LINE_AA)
                    
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                
                cv2.imshow("frame", image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break



In [123]:
keypoints[100]

0.6107305884361267

In [14]:
    cap.release()
    cv2.destroyAllWindows()

In [58]:
!pip install -U scikit-learn scipy matplotlib

data preprocessing

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [16]:
label_map = {label:num for num, label in enumerate(actions)}

In [17]:
label_map

{'hello': 0, 'you': 1, 'how': 2}

In [18]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [19]:
X = np.array(sequences)

In [20]:
X.shape

(90, 30, 1662)

In [21]:
y = to_categorical(labels).astype(int)

In [22]:
y.shape

(90, 3)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

building and training LSTM neural network

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [25]:
log_dir = os.path.join("Logs")
tb_callback = TensorBoard(log_dir=log_dir)

In [26]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [141]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [142]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 3s 78ms/step - loss: 2.8860 - categorical_accuracy: 0.3529
Epoch 2/2000
3/3 [==============================] - 0s 92ms/step - loss: 20.2393 - categorical_accuracy: 0.4000
Epoch 3/2000
3/3 [==============================] - 0s 91ms/step - loss: 8.2300 - categorical_accuracy: 0.3294
Epoch 4/2000
3/3 [==============================] - 0s 85ms/step - loss: 3.8183 - categorical_accuracy: 0.3176
Epoch 5/2000
3/3 [==============================] - 0s 88ms/step - loss: 6.1721 - categorical_accuracy: 0.4118
Epoch 6/2000
3/3 [==============================] - 0s 96ms/step - loss: 3.6975 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 0s 83ms/step - loss: 2.7140 - categorical_accuracy: 0.3529
Epoch 8/2000
3/3 [==============================] - 0s 77ms/step - loss: 4.0649 - categorical_accuracy: 0.2706
Epoch 9/2000
3/3 [==============================] - 0s 85ms/step - loss: 3.5338 - categorical_accuracy: 0.3529


KeyboardInterrupt: 

In [27]:
model.load_weights('finalact1.h5')

In [120]:
model.save('finalact1.h5')

In [49]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [28]:
res = model.predict(x_test)

1/1 [==============================] - 2s 2s/step


In [29]:
actions[np.argmax(res[0])]

'how'

In [30]:
actions[np.argmax(y_test[0])]

'how'

evaluating using confusion matrix 

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [32]:
yhat = model.predict(x_test)

1/1 [==============================] - 0s 84ms/step


In [33]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [34]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

1.0

In [40]:
def speak(s):
    engine.say(s)
    engine.runAndWait()

In [36]:
def pred_sent(sentence):
    s=' '
    if 'hello' in sentence:
        s+='hello,'
    if 'how' and 'you' in sentence:
        s+= ' how are you?'
    return s

real time detection 

In [37]:
sequence = []
sentence = []
predictions = []
threshold = 0.9
qq=0

cap = cv2.VideoCapture('test5.mp4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_det(frame, holistic)
        #print(results)
        
        draw_stlyed_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]
        
        cv2.rectangle(image, (0,0), (640,40), (245,117,16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2,cv2.LINE_AA)   
        cv2.imshow("frame", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 87ms/step
hello
1/1 [==============================] - 0s 40ms/step
hello
1/1 [==============================] - 0s 54ms/step
hello
1/1 [==============================] - 0s 40ms/step
hello
1/1 [==============================] - 0s 45ms/step
hello
1/1 [==============================] - 0s 59ms/step
hello
1/1 [==============================] - 0s 54ms/step
hello
1/1 [==============================] - 0s 61ms/step
hello
1/1 [==============================] - 0s 48ms/step
hello
1/1 [==============================] - 0s 38ms/step
hello
1/1 [==============================] - 0s 41ms/step
hello
1/1 [==============================] - 0s 42ms/step
hello
1/1 [==============================] - 0s 47ms/step
hello
1/1 [==============================] - 0s 47ms/step
hello
1/1 [==============================] - 0s 60ms/step
hello
1/1 [==============================] - 0s 43ms/step
hello
1/1 [==============================] - 0s 39ms/step
hello
1/1 [=========

1/1 [==============================] - 0s 52ms/step
hello
1/1 [==============================] - 0s 51ms/step
hello


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [38]:
    cap.release()
    cv2.destroyAllWindows()

In [117]:
sentence

['hello', 'how', 'you', 'hello']

In [41]:
speak(pred_sent(sentence))